In [1]:
import pandas as pd
import pyodbc

# Define file path
file_path = "C:/Users/1/Desktop/Final Project/Characterizing-Parliament-Members-using-LLM/db_csv/KNS_members_description.xlsx"

# Read the Excel file
df = pd.read_excel(file_path, usecols=["kns_name", "description", "kns_name_hebrew", "knesset_seat"])

# Define database connection parameters
server = "DESKTOP-9LFS960\SQLEXPRESS"  # Change to your server name
database = "KNS_Description"  # Change to your database name

dsn = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes'

# Establish connection
conn = pyodbc.connect(dsn)
cursor = conn.cursor()


In [2]:
df

,kns_name,description,kns_name_hebrew,knesset_seat
0,Moshe Abutbul,"A charismatic and community-driven figure, Mos...",משה אבוטבול,"23,24,25"
1,Yuli Edelstein,"A respected elder statesman within the Likud, ...",יולי אדלשטיין,"14,15,16,17,19,20,21,22,23,24,25"
2,Amir Ohana,"A trailblazer in Israeli politics, Amir Ohana ...",אמיר אוחנה,"20,21,22,23,24,25"
3,Yinon Azulai,"A devoted representative of the Shas party, Yi...",ינון אזולאי,"20,21,22,23,24,25"
4,Gadi Eisenkot,"A revered military leader turned politician, G...",גדי איזנקוט,25
...,...,...,...,...
116,Michal Shir Sagman,Michal Shir-Sagman is an advocate for women’s ...,מיכל שיר סגמן,25
117,Naor Shiri,Naor Shiri is known for his work on labor righ...,נאור שירי,25
118,Osher Shkalim,"A rising political star, Osher Shkalim focuses...",אושר שקלים,25
119,Aida Sliman,Aida Toma-Sliman is a prominent Arab-Israeli p...,עאידה סלימאן,"20,21,22,23,24,25"


In [3]:
# Insert data into kns_history table
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO KNS_Description.dbo.kns_members (kns_name, description, kns_name_hebrew, knesset_seat)
        VALUES (?, ?, ?, ?)""",
        row["kns_name"], row["description"], row["kns_name_hebrew"], row["knesset_seat"]
    )

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

Data inserted successfully!
